In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.layers import Dense, Flatten

### 1. 학습,검증 데이터셋 생성

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  'D:\\ai_phiso\data',
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(150,150))

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  'D:\\ai_phiso\data',
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(150,150))

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
# val_ds에서 이미지와 라벨 추출
xtrain = []
ytrain = []

for images, labels in train_ds:
    xtrain.append(images.numpy())
    ytrain.append(labels.numpy())

# numpy 배열로 변환
xtrain = tf.concat(xtrain, axis=0).numpy()  # 이미지를 하나의 배열로 병합
ytrain = tf.concat(ytrain, axis=0).numpy()  # 레이블을 하나의 배열로 병합

print(f"xtrain shape: {xtrain.shape}, ytrain shape: {ytrain.shape}")

In [ ]:
# val_ds에서 이미지와 라벨 추출
xval = []
yval = []

for images, labels in val_ds:
    xval.append(images.numpy())
    yval.append(labels.numpy())

# numpy 배열로 변환
xval = tf.concat(xval, axis=0).numpy()  # 이미지를 하나의 배열로 병합
yval = tf.concat(yval, axis=0).numpy()  # 레이블을 하나의 배열로 병합

print(f"xval shape: {xval.shape}, yval shape: {yval.shape}")

### 2. 데이터 증강

In [ ]:
# InceptionV3, 자체 생성 모델의 경우 해당 데이터셋 사용
datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=10,  
    zoom_range=0.2,  
    width_shift_range=0.1,  
    height_shift_range=0.1, 
    horizontal_flip=True,  
    vertical_flip=False
)

In [ ]:
# MobileNet의 경우 -1 ~ 1의 범위로 정규화가 필요하기에 스케일링을 제외함
datagen2 = ImageDataGenerator(
    rotation_range=10,  
    zoom_range=0.2, 
    width_shift_range=0.1,  
    height_shift_range=0.1, 
    horizontal_flip=True,  
    vertical_flip=False
)  

In [ ]:
img_iter = datagen.flow(xtrain, ytrain)
img_iter2 = datagen2.flow(xtrain, ytrain)

### 3. 자체 모델 생성

- 34.58%의 정확도 달성

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(150, (3, 3), activation='relu', input_shape=(150,150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(75, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(75, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(37, activation='relu'))
model.add(layers.Dense(6,activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'],
              optimizer=keras.optimizers.Adam(learning_rate=0.001))

esc = EarlyStopping(monitor='val_loss', patience=3)

h = model.fit(img_iter,
          epochs=20,
          validation_data=(xval, yval),
          callbacks = [esc])

### 4. MobileNet 파인 튜닝

- 95.67%의 정확도 달성

In [ ]:
mobilenet = MobileNetV3Large()
mobilenet.trainable = False

inputs = tf.keras.Input(shape=(150,150,3))
x = mobilenet(inputs, training=False)
x = Flatten()(x)
output = Dense(6, activation='softmax')(x)

mobilenet = tf.keras.Model(inputs = inputs, outputs=output)
mobilenet.summary()

# 모델 컴파일

mobilenet.compile(
    loss='sparse_categorical_crossentropy', # 레이블이 문자열인 경우 sparse 사용
    metrics=['accuracy'],
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
)

# 콜백 설정
es = EarlyStopping(monitor='val_loss', verbose=1, patience=3)

# 모델 학습
history = mobilenet.fit(
    img_iter2,
    validation_data=(xval, yval),
    epochs=40,
    callbacks=[es]
)

In [ ]:
# MobileNet 학습과정 시각화

plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history.history['loss'], 'y', label = 'train loss')
plt.plot(history.history['val_loss'], 'r', label = 'val loss')
plt.title('MobileNetV3Large Loss')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], 'y', label = 'train accuracy')
plt.plot(history.history['val_accuracy'], 'r', label = 'val accuracy')
plt.title('MobileNetV3Large Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
model_save_path = 'mobilenet_custom_trained_model.h5'
mobilenet.save(model_save_path)

### 5. InceptionV3 파인 튜닝

- 97%의 정확도 달성

In [ ]:
incep = InceptionV3(include_top=False)
incep.trainable = False

inputs = tf.keras.Input(shape=(150,150,3))
x = incep(inputs, training=False)
x = Flatten()(x)
output = Dense(6, activation='softmax')(x)

incep = tf.keras.Model(inputs = inputs, outputs=output)

# 모델 컴파일

incep.compile(
    loss='sparse_categorical_crossentropy', # 레이블이 문자열인 경우 sparse 사용
    metrics=['accuracy'],
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
)

# 콜백 설정
es = EarlyStopping(monitor='val_loss', verbose=1, patience=3)

# 모델 학습
incep_history = incep.fit(
    img_iter2,
    validation_data=(xval, yval),
    epochs=20,
    callbacks=[es]
)